In [ ]:
"""
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-1d71bac5-a341-449f-9ca7-b29de95e771a',
    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'api_key': 'ejujKveBRS3Bk7l3pzjemEQDhTSaOKmqCg6x6osXWkx4'
}

configuration_name = 'os_2d970470a6354234a1716fe9f4db519b_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('US_Accidents_Dec19.csv', 'bdppproject-donotdelete-pr-nejualq57kuqe4'))
"""

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

try:
    spark.close()
except: 
    pass

try:
    conf = SparkConf().setAppName('Spark Project')
    sc = SparkContext(conf=conf)
except Exception as e:
    print(e)
#.config("spark.some.config.option", "some-value") \
#SparkContext.setSystemProperty('spark.executor.memory', '8g')
#spark = SparkSession \
#    .builder \
#    .appName("Spark Project") \
#    .config("spark.executor.memory", '5g') \
#    .config("spark.driver.memory", '5g') \
#    .getOrCreate()

spark = SparkSession.builder \
    .appName("Spark Project") \
    .master("local[*]") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.memory", "70g") \
    .config("spark.driver.memory", "12g") \
    .config("spark.memory.offHeap.enabled",True) \
     .config("spark.memory.offHeap.size","16g")  \
    .config("spark.yarn.executor.memoryOverhead",200) \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.default.parallelism", "6") \
    .getOrCreate()

df = spark.read.load("data/US_Accidents_Dec19.csv",format="csv", sep=",", inferSchema="true", header="true")

In [3]:
sc

<SparkContext master=local[*] appName=Spark Project>

In [4]:
sc._conf.getAll()

[('spark.driver.port', '54834'),
 ('spark.app.id', 'local-1588170628522'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.host', 'Damon'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'Spark Project'),
 ('spark.ui.showConsoleProgress', 'true')]

# Imports

In [5]:
from pyspark.sql.functions import when, count, col, isnan, countDistinct,from_unixtime,from_utc_timestamp, unix_timestamp,split, to_timestamp, hour, month, lit,collect_list
import seaborn as sns 
import matplotlib.pyplot as plt 

# Due to problem in spark 3.0.0
spark.conf.set("spark.sql.legacy.utcTimestampFunc.enabled", "true")

# Define variables

In [6]:
# Define data information

# Choose directory
#projDir = "C:\Users\fredr\Documents\github-projects\BDPP-Labs\Project\Models"


# Numerical values

colLabel = ["Severity"]

colRem = ['ID', 
          'Source',
          'End_Time',
          'End_Lat',
          'End_Lng',
          'Description',
        ]

# Dropping data that cant help during model
df = df.drop(*colRem)

# Convert boolean to string since PCA cant handle boolean which should be a class
df = df.select(*[col(c[0]).cast("string").alias(c[0]) if c[1] == 'boolean' else col(c[0]).alias(c[0]) for c in df.dtypes])

#renamedHousing.select([count(when(col(c).isNull(), c)).alias(c) for c in colNum]).show()
rdd = sc.parallelize(df.dtypes)

colCat = rdd.map(lambda i: i[0] if (i[1]=='string' or i[1]=='boolean' and i[0]) else None).filter(lambda i: i != None).collect()
colNum = rdd.map(lambda i: i[0] if (i[1]=='double' and i[0]) else None).filter(lambda i: i != None).collect()

print(colCat)
print(colNum)

['Start_Time', 'Street', 'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
['TMC', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Number', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']


# 2. Preprocessing

## Analyse single city
This could be CA since it stands for a little less than 50% of the total set

In [7]:
df = df.filter(df.State == "CA") # Lowers the dataset quite a lot

## Modify time
Convert the timestamp into a numeric value and then into a string so that the time of day and month can be categorised.

In [8]:
# Convert to int then cast to string

df = df.withColumn('Start_Hour', hour(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Start_Month', month(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df = df.withColumn('Weather_Hour', hour(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Weather_Month', month(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df.show(1)

+-----+--------+-------------------+---------+-----------+------------+------+-----------+----+-------+------+-----+-------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+----------+-----------+------------+-------------+
|  TMC|Severity|         Start_Time|Start_Lat|  Start_Lng|Distance(mi)|Number|     Street|Side|   City|County|State|Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity| Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station| Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|Sun

## Cast data
Cast all the datacolumns into correct format so they will be sorted to numerical or categorical values.

In [9]:
# Convert from string to integer values
# TODO: Does not work since data contains Nan/Null?
df = df.select(
        col('TMC').cast('string'),
        col('Severity').cast('int'),
        #col('Start_Time').cast('int'),
        col('Start_Hour').cast('string'),
        col('Start_Month').cast('string'),
        col('Weather_Hour').cast('string'),
        col('Weather_Month').cast('string'),
        #col('Start_Lat').cast('string'),
        #col('Start_Lng').cast('string'),
        col('Distance(mi)').cast('double'),
        col('Street').cast('string'),       # Remove eventually?
        col('Side').cast('string'),         # Remove eventually?
        col('City').cast('string'),         # Remove eventually?
        col('State').cast('string'),        # Remove eventually?
        #col('Zipcode').cast('string'),      # Remove eventually?
        #col('Country').cast('string'),      # Remove eventually?
        #col('Airport_Code').cast('string'), # Remove eventually?
      #  col('Weather_Timestamp').cast('int'),
        col('Temperature(F)').cast('double'),
        col('Wind_Chill(F)').cast('double'),
        col('Humidity(%)').cast('double'),
        col('Pressure(in)').cast('double'),
        col('Visibility(mi)').cast('double'),
        col('Wind_Direction').cast('string'),
        col('Wind_Speed(mph)').cast('double'),
        col('Weather_Condition').cast('string'),
        col('Amenity').cast('string'),
        col('Bump').cast('string'),
        col('Crossing').cast('string'),
        col('Give_Way').cast('string'),
        col('Junction').cast('string'),
        col('No_Exit').cast('string'),
        col('Railway').cast('string'),
        col('Roundabout').cast('string'),
        col('Station').cast('string'),
        col('Stop').cast('string'),
        col('Traffic_Calming').cast('string'),
        col('Traffic_Signal').cast('string'),
        col('Turning_Loop').cast('string'),
        col('Sunrise_Sunset').cast('string'),
        col('Civil_Twilight').cast('string'),
        col('Nautical_Twilight').cast('string'),
        col('Astronomical_Twilight').cast('string')
    ) 

# Since the last check can be unorganised we recreate a new list that contains all data
colLabel = ["Severity"]

rdd = sc.parallelize(df.dtypes)
colCat = rdd.map(lambda i: i[0] if (i[1]=='string' or i[1]=='boolean' and i[0] not in colLabel) else None).filter(lambda i: i != None).collect()
colNum = rdd.map(lambda i: i[0] if (i[1]=='double' and i[0] not in colLabel) else None).filter(lambda i: i != None).collect()

In [10]:
print(colLabel)
print(colCat)
print(colNum)

['Severity']
['TMC', 'Start_Hour', 'Start_Month', 'Weather_Hour', 'Weather_Month', 'Street', 'Side', 'City', 'State', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
['Distance(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']


## Remove data containing NaN (Drop)
Since NaN and Null occurs we drop these rows

In [11]:
df.count()
df = df.na.drop()
df.count()

89563

### Recheck the missing values
Check so that the output contains 0 missing values

In [12]:
df_missingVals_cols = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
# Missing value in each column
df_missingVals_cols.collect()

[Row(TMC=0, Severity=0, Start_Hour=0, Start_Month=0, Weather_Hour=0, Weather_Month=0, Distance(mi)=0, Street=0, Side=0, City=0, State=0, Temperature(F)=0, Wind_Chill(F)=0, Humidity(%)=0, Pressure(in)=0, Visibility(mi)=0, Wind_Direction=0, Wind_Speed(mph)=0, Weather_Condition=0, Amenity=0, Bump=0, Crossing=0, Give_Way=0, Junction=0, No_Exit=0, Railway=0, Roundabout=0, Station=0, Stop=0, Traffic_Calming=0, Traffic_Signal=0, Turning_Loop=0, Sunrise_Sunset=0, Civil_Twilight=0, Nautical_Twilight=0, Astronomical_Twilight=0)]

### Remove data with occurance less than 1%
Based on information from analysis. With further analysis lower procentage can be used to find better results.

In [13]:
weather_freq = df.groupBy('Weather_Condition').count().orderBy('count',ascending=False)

n = int(df.count()*0.001) # Limit the plot to ignore conditions below an limit

df_filtered = weather_freq.filter(weather_freq['count'] > n)
df_filtered.show()
df.count()

+--------------------+-----+
|   Weather_Condition|count|
+--------------------+-----+
|                Fair|42320|
|              Cloudy|13231|
|       Partly Cloudy| 9610|
|       Mostly Cloudy| 8372|
|               Clear| 5317|
|          Light Rain| 2626|
|                Haze| 2381|
|                 Fog| 1351|
|            Overcast| 1128|
|                Rain|  833|
|        Fair / Windy|  523|
|    Scattered Clouds|  463|
|          Heavy Rain|  283|
|               Smoke|  189|
|          Light Snow|  125|
|Partly Cloudy / W...|  110|
|Mostly Cloudy / W...|   93|
+--------------------+-----+



89563

In [14]:
filtered_conditions = df_filtered.select("Weather_Condition").rdd.flatMap(lambda x: x).collect()

print(filtered_conditions)

test1 = df.filter(df['Weather_Condition'].isin(*filtered_conditions))

df = test1 # Adding the filtered data to dataframe (Hopefully we lose less than n% of the data) but reduce the dimension of feature vector
df.count()

['Fair', 'Cloudy', 'Partly Cloudy', 'Mostly Cloudy', 'Clear', 'Light Rain', 'Haze', 'Fog', 'Overcast', 'Rain', 'Fair / Windy', 'Scattered Clouds', 'Heavy Rain', 'Smoke', 'Light Snow', 'Partly Cloudy / Windy', 'Mostly Cloudy / Windy']


88955

## Prepare Pipeline

### Quantiles function
Alternative solution to get position (Work in progress)

In [15]:
# Remove City, Country, State, Zipcode, Airport_Code
# In order to reduce dimensionality a kernel function will be applied upon the Start_Lat and Start_Lng then fused in order to create a new set of data
# Start_Lat
# Start_Lng

from pyspark.ml.feature import QuantileDiscretizer
"""
dscretizer_Lat = QuantileDiscretizer(numBuckets=50, inputCol="Start_Lat", outputCol="Start_Lat")
dscretizer_Lng = QuantileDiscretizer(numBuckets=50, inputCol="Start_Lng", outputCol="Start_Lng")

# Convert into categorical values (two blocks)
quant_Lat_model = quantiles_Lat.fit(df_features)
quant_Lng_model = quantiles_Lng.fit(df_features) 

quanted_Lat = quant_Lat_model.transform(df_features)
quanted_Lng = quant_Lng_model.transform(df_features)

# Into categorical values
stringIndexer = StringIndexer(inputCol=["Start_Lat","Start_Lng"], outputCol=["Start_Lat","Start_Lng"] + "_num")
indexer_model = stringIndexer.fit(df_features) # Change into Quanted_Lat, Quanted_Lng
indexed = indexer_model.transform(df_features)

# One-hot 
encoder = OneHotEncoder(inputCols=["Start_Lat_num","Start_Lng_num"], outputCols=[s + "_vec" for s in colCat])

encoded = encoder.fit(indexed)
"""

# Do multiplication into a final feature vector (Kernel trick)

# Remove City, Country, State, Zipcode, Airport_Code

'\ndscretizer_Lat = QuantileDiscretizer(numBuckets=50, inputCol="Start_Lat", outputCol="Start_Lat")\ndscretizer_Lng = QuantileDiscretizer(numBuckets=50, inputCol="Start_Lng", outputCol="Start_Lng")\n\n# Convert into categorical values (two blocks)\nquant_Lat_model = quantiles_Lat.fit(df_features)\nquant_Lng_model = quantiles_Lng.fit(df_features) \n\nquanted_Lat = quant_Lat_model.transform(df_features)\nquanted_Lng = quant_Lng_model.transform(df_features)\n\n# Into categorical values\nstringIndexer = StringIndexer(inputCol=["Start_Lat","Start_Lng"], outputCol=["Start_Lat","Start_Lng"] + "_num")\nindexer_model = stringIndexer.fit(df_features) # Change into Quanted_Lat, Quanted_Lng\nindexed = indexer_model.transform(df_features)\n\n# One-hot \nencoder = OneHotEncoder(inputCols=["Start_Lat_num","Start_Lng_num"], outputCols=[s + "_vec" for s in colCat])\n\nencoded = encoder.fit(indexed)\n'

### Impute 

In [16]:
# Probobly a bad Idea but why not?
from pyspark.ml.feature import Imputer

#imputer = Imputer(inputCols=['Wind_Chill(F)', 'Wind_Speed(mph)'], outputCols=['Wind_Chill(F)', 'Wind_Speed(mph)'])
imputer = Imputer(inputCols=colNum, outputCols=colNum)
imputer.setStrategy("median")

Imputer_c5669186edb7

### Scaling

In [17]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

#Create a vector
num_assembler = VectorAssembler(inputCols=colNum, outputCol="num_features",handleInvalid="keep")
scaler = StandardScaler(inputCol="num_features", withMean=True, withStd=True, outputCol="scaledFeatures")
scaler

StandardScaler_b06111df4d93

### Indexer

In [18]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
#colCat
#col_assembler = VectorAssembler(inputCols=colCat, outputCol="cat_features",handleInvalid="keep")
#indexers = StringIndexer(inputCol="cat_features", outputCol="indexedFeatures",handleInvalid="keep")

indexers = [StringIndexer(inputCol = c, outputCol = c +'_IDX', handleInvalid='keep') for c in colCat]
print(indexers)

[StringIndexer_664a66fc11b3, StringIndexer_f21596f5353b, StringIndexer_2b3aebc0731a, StringIndexer_8f95b76ef58b, StringIndexer_d2165564a0d8, StringIndexer_f4295b6ecee4, StringIndexer_4d0abdcfadef, StringIndexer_ab8365ebad81, StringIndexer_94cae7a421fb, StringIndexer_a92712581716, StringIndexer_83ab83f9cbe3, StringIndexer_6e1faf10d558, StringIndexer_2258dd4edce5, StringIndexer_1a96a7b88f31, StringIndexer_a70126bec3e8, StringIndexer_d7249e6673ee, StringIndexer_fc3a744b85da, StringIndexer_af251ec28c35, StringIndexer_b7025f849367, StringIndexer_d6d056bc5704, StringIndexer_d8466531547f, StringIndexer_d040642f09ae, StringIndexer_2cc7b1187f24, StringIndexer_bc605b89a4a6, StringIndexer_3bf9b61538bb, StringIndexer_9ba103505f2b, StringIndexer_f10ad26f175e, StringIndexer_d7d2be2df76d]


In [19]:
# Debug
#from pyspark.ml import Pipeline
#pipeline = Pipeline(stages=indexers)
#df_indexers = pipeline.fit(df).transform(df)

#col_assembler = VectorAssembler(inputCols=[c +'_IDX' for c in colCat], outputCol="features",handleInvalid="keep")
#data = col_assembler.transform(df_indexers)
#df_features.withColumn('features', col(col_assembler.transform(df_indexers)))
#df_indexers.select([c +'_IDX' for c in colCat]).show(1)

### Encoder

In [20]:
from pyspark.ml.feature import OneHotEncoder 

#ohe = OneHotEncoder()
#ohe.setInputCols([c + "_IDX" for c in colCat])
#ohe.setOutputCols([c + "_IDX_vec" for c in colCat])

#col_assembler = VectorAssembler(inputCols=[c + "_IDX" for c in colCat], outputCol="col_features",handleInvalid="keep")
encoder = OneHotEncoder(inputCols=[c + "_IDX" for c in colCat], outputCols=[c + "_IDX_vec" for c in colCat],handleInvalid="keep")
#encoder = OneHotEncoder(inputCol="col_features", outputCol="col_features_vec",handleInvalid="keep")
print(encoder)

OneHotEncoder_34ed03483f16


In [21]:
# Debug
#from pyspark.ml import Pipeline 
#pipeline = Pipeline(stages=[*indexers,*encoder])
#df_indexers = pipeline.fit(df).transform(df)
#df_indexers.select([c +'_IDX_vec' for c in colCat]).show(1)

In [22]:
df.show(1)

+-----+--------+----------+-----------+------------+-------------+------------+----------+----+------------+-----+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|  TMC|Severity|Start_Hour|Start_Month|Weather_Hour|Weather_Month|Distance(mi)|    Street|Side|        City|State|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Weather_Condition|Amenity| Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station| Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|Sunrise_Sunset|Civil_Twilight|Nautical_Twilight|Astronomical_Twilight|
+-----+--------+----------+-----------+------------+-------------+------------+----------+----+------------+-----+--------------+-------------

## Count categorical values
Confirm the amount of categories

In [23]:
df.agg(*(countDistinct(col(c)).alias(c) for c in colCat)).collect()[0].asDict()

{'TMC': 19,
 'Start_Hour': 24,
 'Start_Month': 12,
 'Weather_Hour': 24,
 'Weather_Month': 12,
 'Street': 9096,
 'Side': 2,
 'City': 967,
 'State': 1,
 'Wind_Direction': 23,
 'Weather_Condition': 17,
 'Amenity': 2,
 'Bump': 2,
 'Crossing': 2,
 'Give_Way': 2,
 'Junction': 2,
 'No_Exit': 2,
 'Railway': 2,
 'Roundabout': 2,
 'Station': 2,
 'Stop': 2,
 'Traffic_Calming': 2,
 'Traffic_Signal': 2,
 'Turning_Loop': 1,
 'Sunrise_Sunset': 2,
 'Civil_Twilight': 2,
 'Nautical_Twilight': 2,
 'Astronomical_Twilight': 2}

## Preprocessing - Pipeline

In [27]:
# Categorical values

from pyspark.ml import Pipeline

numPipeline = Pipeline(stages=[imputer, num_assembler, scaler])
catPipeline = Pipeline(stages=[*indexers, encoder])

#.select("scaledFeatures", *[c +'_IDX_vec' for c in colCat])
pipeline = Pipeline(stages=[numPipeline, catPipeline])
preprocessed_df = pipeline.fit(df).transform(df)
#print("Saving model")
#preprocessed_df.save(projDir + "\pipeline")
#print("Saved complete")

In [28]:
#preprocessed_df = preprocessed_df.transform(df)

In [29]:
#catPipeline.fit(df).transform(df).show(1)

In [30]:
preprocessed_df.select("scaledFeatures",*[c + "_IDX_vec" for c in colCat]).show(2,False)

+----------------------------------------------------------------------------------------------------------------------------------------------+--------------+------------------+-------------------+--------------------+---------------------+-------------------+-------------+-----------------+-------------+----------------------+-------------------------+---------------+-------------+----------------+----------------+----------------+---------------+---------------+------------------+---------------+-------------+-----------------------+----------------------+--------------------+----------------------+----------------------+-------------------------+-----------------------------+
|scaledFeatures                                                                                                                                |TMC_IDX_vec   |Start_Hour_IDX_vec|Start_Month_IDX_vec|Weather_Hour_IDX_vec|Weather_Month_IDX_vec|Street_IDX_vec     |Side_IDX_vec |City_IDX_vec     |State_IDX_vec|Wind

## Combine features

In [31]:
va2 = VectorAssembler(inputCols=["scaledFeatures",*[c + "_IDX_vec" for c in colCat]], outputCol="final_features")

temp1 = va2.transform(preprocessed_df)
temp1.show(1)
temp1 = temp1.withColumn('label', col("Severity"))
dataset = temp1.withColumn('features', temp1.final_features).select("features","label")
dataset.show(2, False)

+-----+--------+----------+-----------+------------+-------------+------------+----------+----+------------+-----+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+--------------------+--------------------+-------+--------------+---------------+----------------+-----------------+----------+--------+--------+---------+------------------+---------------------+-----------+--------+------------+------------+------------+-----------+-----------+--------------+-----------+--------+-------------------+------------------+----------------+------------------+------------------+---------------------+-------------------------+--------------------+----------------+-----------------------------+----------------------+-----------------------

# 3. Feature importance

In [32]:
# ChiSqSelector
from pyspark.ml.feature import ChiSqSelector

selector = ChiSqSelector(numTopFeatures=40, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="label")

result = selector.fit(dataset).transform(dataset)
print("ChiSqSelector output with top %d features selected" % selector.getNumTopFeatures())
result.show()


ChiSqSelector output with top 1 features selected
+--------------------+-----+--------------------+
|            features|label|    selectedFeatures|
+--------------------+-----+--------------------+
|(10265,[0,1,2,3,4...|    2|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    2|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    2|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    2|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    2|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    2|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    3|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    2|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    3|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    2|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    3|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    2|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    2|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    2|(1,[0],[-0.130729...|
|(10265,[0,1,2,3,4...|    3|(1,[0],[-0.115437...|
|(10265,[0,1,2,3,4...|    3|(1,[0],[-0.115437...|


In [36]:
result.show(1,False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+------------------------------+
|features                                                                                                                                                                                                                                                                                                                                                                                                                                   |label|selectedFeatures              |
+-----------------------------------------------------------------

# 4. Machine learning

In [33]:
trainSet, testSet = dataset.randomSplit([0.8 ,0.2], 1)

In [34]:
trainSet.show(2,False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                                                                                                                                                                |label|
+-------------------------------------------------------------------------------------------------------------------------------------

In [37]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="label",maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(trainSet)
trainingSummary = lrModel.summary
print(trainingSummary)
#print("Coefficients: %s" % str(lrModel.coefficients))
#print("Intercept: %s" % str(lrModel.intercept))
#print("RMSE: %s" % str(trainingSummary.rootMeanSquaredError))

In [ ]:
trainingSummary.accuracy